<a href="https://colab.research.google.com/github/KoNGoL/tfm_breast_cancer_detection/blob/main/Retinanet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!mkdir -p /content/Drive/
from google.colab import drive
drive.mount('/content/Drive') 
!mkdir -p /content/Drive/MyDrive/RetinaNet/
%cd /content/Drive/MyDrive/RetinaNet/

Mounted at /content/Drive
/content/Drive/MyDrive/RetinaNet


In [2]:
# !git clone https://github.com/fizyr/keras-retinanet
%cd /content/Drive/MyDrive/RetinaNet/keras-retinanet/
!pip install numpy --user
!pip install . --user
!python setup.py build_ext --inplace
# !wget https://github.com/fizyr/keras-retinanet/releases/download/0.5.1/resnet50_coco_best_v2.1.0.h5
# !mv  /content/Drive/MyDrive/RetinaNet/keras-retinanet/resnet50_coco_best_v2.1.0.h5  /content/Drive/MyDrive/RetinaNet//keras-retinanet/snapshots/


/content/Drive/MyDrive/RetinaNet/keras-retinanet
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing /content/Drive/MyDrive/RetinaNet/keras-retinanet
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Created wheel for keras-retinanet: filename=keras_retinanet-1.0.0-cp37-cp37m-linux_x86_64.whl size=169457 sha256=e04bfdf148a5244882db54d25fb9afbd1cfce69c5783a922776762d2a0e9d686
  Stored in directory: /root/.cache/pip/wheels/be/04/af/1c7daf7a6159b5274267d4629d60ed538b69299ec9996cc17f
  Created 

In [ ]:
%cd examples 
!python resnet50_retinanet.py

/content/keras-retinanet/examples
Traceback (most recent call last):
  File "resnet50_retinanet.py", line 31, in <module>
    setup_gpu(gpu)
  File "../keras_retinanet/utils/gpu.py", line 22, in setup_gpu
    visible_gpu_indices = [int(id) for id in gpu_id.split(',')]
AttributeError: 'int' object has no attribute 'split'


In [ ]:
%cd /content
!pip install roboflow

from roboflow import Roboflow
# rf = Roboflow(api_key="yGM3vBhGiZQUJrW3szSn")
# project = rf.workspace("tfm-pq95g").project("tfm-breast_640_separated")
# dataset = project.version(1).download("voc")

In [ ]:
!mkdir /content/Drive/MyDrive/RetinaNet/dataset
!mkdir /content/Drive/MyDrive/RetinaNet/dataset/{Annotations,ImageSets,JPEGImages}
!mkdir /content/Drive/MyDrive/RetinaNet/dataset/ImageSets/Main
!cp /content/TFM-breast_640_separated-1/test/*jpg /content/Drive/MyDrive/RetinaNet/dataset/JPEGImages/
!cp /content/TFM-breast_640_separated-1/train/*jpg /content/Drive/MyDrive/RetinaNet/dataset/JPEGImages/
!cp /content/TFM-breast_640_separated-1/valid/*jpg /content/Drive/MyDrive/RetinaNet/dataset/JPEGImages/
!cp /content/TFM-breast_640_separated-1/train/*xml /content/Drive/MyDrive/RetinaNet/dataset/Annotations/
!cp /content/TFM-breast_640_separated-1/test/*xml /content/Drive/MyDrive/RetinaNet/dataset/Annotations/
!cp /content/TFM-breast_640_separated-1/valid/*xml /content/Drive/MyDrive/RetinaNet/dataset/Annotations/

In [ ]:
from os import listdir
from os.path import isfile, join

# dataset_directory = "/content/dataset/Annotations"

with open("/content/Drive/MyDrive/RetinaNet/dataset/ImageSets/Main/test.txt", "w") as test:
    for file in listdir('/content/TFM-breast_640_separated-1/test/'):
        if ".xml" in file:
          continue
        name = file.split("\\")[-1]
        name = name.replace('.jpg', '')
        test.write(name + "\n")
        # print("test:    " + name)
        
with open("/content/Drive/MyDrive/RetinaNet/dataset/ImageSets/Main/train.txt", "w") as train:
    for file in listdir('/content/TFM-breast_640_separated-1/train/'):
        if ".xml" in file:
          continue
        name = file.split("\\")[-1]
        name = name.replace('.jpg', '')
        train.write(name + "\n")   
        # print("train:    " + name)   

with open("/content/Drive/MyDrive/RetinaNet/dataset/ImageSets/Main/val.txt", "w") as val:
    for file in listdir('/content/TFM-breast_640_separated-1/valid/'):
        if ".xml" in file:
          continue
        name = file.split("\\")[-1]
        name = name.replace('.jpg', '')
        val.write(name + "\n")   
        # print("train:    " + name)

test.close()
train.close()
val.close()

In [ ]:
# !cp -r /content/Drive/MyDrive/RetinaNet/dataset /content/

#empezar entrenamiento
# !python /content/Drive/MyDrive/RetinaNet/keras-retinanet/keras_retinanet/bin/train.py --imagenet-weights --gpu 0 --epoch 100 --batch-size 8 --steps 614 --snapshot-path /content/Drive/MyDrive/RetinaNet/retinanet_models/ --tensorboard-dir /content/Drive/MyDrive/RetinaNet/retinanet_log/ pascal /content/Drive/MyDrive/RetinaNet/dataset/
#continuar entrenamiento
!python /content/Drive/MyDrive/RetinaNet/keras-retinanet/keras_retinanet/bin/train.py --initial-epoch 48 --snapshot /content/Drive/MyDrive/RetinaNet/retinanet_models/resnet50_pascal_48.h5 --gpu 0 --epoch 100 --batch-size 8 --steps 614 --snapshot-path /content/Drive/MyDrive/RetinaNet/retinanet_models/ --tensorboard-dir /content/Drive/MyDrive/RetinaNet/retinanet_log/ pascal /content/dataset/

backbone
check_tf_version
1 Physical GPUs, 1 Logical GPUs
setup_gpu
read_config_file
PASCAL
PASCAL2
PASCAL3
create_generators
Loading model, this may take a second...
Model: "retinanet"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None, None,  0           []                               
                                 3)]                                                              
                                                                                                  
 conv1 (Conv2D)                 (None, None, None,   9408        ['input_1[0][0]']                
                                64)                                                               
                                                                                                  
 bn_conv1 (BatchNormal